## AI Agent智能应用从0到1定制开发 
## AI Agent Intelligent Application Custom Development from 0 to 1
******
- 此代码为网课《AI Agent智能应用从0到1定制开发》的配套代码，需要注意本套代码建议与网课适配配合食用。
- This code for the online course <AI Agent Intelligent Applications from 0 to 1 custom development> supporting code, need to pay attention to this set of code is recommended with the online course adapted to work with consumption.
- 需要注意由于课程开发周期的原因，langchain版本跨越了3个大版本，部分代码会与视频演示有差别!
- Note that due to the course development cycle, the langchain version spans 3 major releases and some of the code will differ from the video demo!
- 课程地址：https://coding.imooc.com/class/822.html
- Course address: https://coding.imooc.com/class/822.html

### 从环境变量中读取密钥
### Read the key from the environment variable
- 注意：尽量将你的OpenAI Key存储在类似.env文件中，而不是明文暴露在代码里，这是一种基本的安全措施
- Note: Try to store your OpenAI Key in something like an .env file, rather than exposing it explicitly in code, as a basic safety measure!
******

In [1]:

import os
from dotenv import load_dotenv
# Load environment variables from openai.env file
load_dotenv("asset/openai.env")

# Read the OPENAI_API_KEY from the environment
api_key = os.getenv("OPENAI_API_KEY")
api_base = os.getenv("OPENAI_API_BASE")
os.environ["OPENAI_API_KEY"] = api_key
os.environ["OPENAI_API_BASE"] = api_base
os.environ["SERPAPI_API_KEY"] = os.getenv("SERPAPI_API_KEY")
os.environ["ELEVEN_API_KEY"] = os.getenv("ELEVEN_API_KEY")
os.environ["AZURE_COGS_KEY"] = os.getenv("AZURE_COGS_KEY")
os.environ["AZURE_COGS_ENDPOINT"] = os.getenv("AZURE_COGS_ENDPOINT")
os.environ["AZURE_COGS_REGION"] = os.getenv("AZURE_COGS_REGION")

### Prompt+LLM
基本构成： 
PromptTemplate / ChatPromptTemplate -> LLM / ChatModel -> OutputParser
### Prompt+LLM 
Basic components: 
PromptTemplate / ChatPromptTemplate -> LLM / ChatModel -> OutputParser

In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template("给我讲一个关于{foo}的笑话")
model = ChatOpenAI(
    temperature=0,
    model="gpt-3.5-turbo",
)
chain = prompt | model

In [3]:
chain.invoke({"foo": "狗熊"})

AIMessage(content='好的，这里有一个关于狗熊的笑话：\n\n为什么狗熊不喜欢在雨天出去玩耍？\n\n因为它怕变成“湿熊”！哈哈哈！', response_metadata={'token_usage': {'completion_tokens': 65, 'prompt_tokens': 23, 'total_tokens': 88, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-141f3f59-be3e-4588-8648-384fa5c9d619-0')

- 自定义停止输出符
- Custom Stop Output Character

In [4]:
chain = prompt | model.bind(stop=["\n"])

In [5]:
chain.invoke({"foo": "狗熊"})

AIMessage(content='好的，这是一个关于狗熊的笑话：', response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 23, 'total_tokens': 41, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-70fe9aec-6ed7-48bc-bf19-d0404abc1fbd-0')

- 兼容openai函数调用的方式
- Compatible with openai function calls

In [6]:
functions = [
    {
        "name": "joke",
        "description": "讲笑话",
        "parameters": {
            "type": "object",
            "properties": {
                "setup": {"type": "string", "description": "笑话的开头"},
                "punchline": {
                    "type": "string",
                    "description": "爆梗的结尾",
                },
            },
            "required": ["setup", "punchline"],
        },
    }
]
chain = prompt | model.bind(function_call={"name": "joke"}, functions=functions)

In [7]:
chain.invoke({"foo": "男人"}, config={})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"setup":"为什么男人喜欢开车？","punchline":"因为开车时他们可以控制方向盘，而在家里他们控制不了任何东西。"}', 'name': 'joke'}}, response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 90, 'total_tokens': 144, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-19b0536f-ed48-4b92-a5ce-e8bb5460ec56-0')

- 输出解析器
- Output parser

In [8]:
from langchain_core.output_parsers import StrOutputParser

chain = prompt | model | StrOutputParser()

In [9]:
chain.invoke({"foo": "女人"})

'好的，这是一个关于女人的笑话：\n\n为什么女人常常忘记关上柜子？因为她们总是把自己的身体放在第一位，所以她们总是“关心”着自己。哈哈哈哈。'

- 与函数调用混合使用
- Mixed use with function calls

In [10]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

chain = (
    prompt
    | model.bind(function_call={"name": "joke"}, functions=functions)
    | JsonOutputFunctionsParser()
)

In [11]:
chain.invoke({"foo": "女人"})

{'punchline': '因为她不喜欢男人插手她的事情！', 'setup': '为什么女人不喜欢男人指手画脚？'}

In [12]:
#只输出setup
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

chain = (
    {"foo": RunnablePassthrough()} #使用RunnablePassthrough()跳过prompt
    | prompt
    | model.bind(function_call={"name": "joke"}, functions=functions)
    | JsonKeyOutputFunctionsParser(key_name="punchline") # 定义输出的key
)

In [13]:
#chain.invoke({"foo": "女人"})
#使用RunnablePassthrough()跳过prompt
chain.invoke("女人")

'因为商场有很多"排除万难"的东西！'